In [23]:
from dotenv import load_dotenv
import os
GIT_TOKEN = os.getenv("GIT_TOKEN")
load_dotenv()



True

#### State 설정

In [2]:
from typing import Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI

class AgentState(TypedDict):
    """The state of our agent."""
    question: str
    certainty_score: int
    search_results: list
    web_score: str
    repo_name: str
    generation: str

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)



#### 질문에 대한 LLM의 답변 신뢰도 점검

In [3]:
from pydantic import BaseModel, Field

def check_certainty(state: AgentState) -> AgentState:
    """Evaluate certainty score for the query."""
    question = state["question"]

    class CertaintyScoreResponse(BaseModel):
        score: int = Field(description="Certainty score from 1 to 100. Higher is better.")

    certainty_score = llm.with_structured_output(CertaintyScoreResponse)

    print("--- Checking LLM's Certainty ---")
    score_response = certainty_score.invoke(question)

    return {
        "certainty_score": score_response.score
    }

In [4]:
def route_based_on_certainty(state: AgentState) -> Literal["web_search", "direct_response"]:
    """Route to appropriate node based on certainty score."""
    score = state["certainty_score"]

    if score != 100:
        print("--- LLM is not certain so It will do web Search ---")
        return "web_search"
    else:
        print("--- LLM is certain so It will generate answer directly ---")
        return "direct_response"

In [5]:
question = "Langgraph로 rag를 구축하는 방법"

score_print = check_certainty({"question": question})
print(score_print)


--- Checking LLM's Certainty ---
{'certainty_score': 85}


#### LLM이 스스로 답변할 수 있는 경우의 노드

In [6]:
def direct_response(state: AgentState):
    question = state["question"]
    result = llm.invoke(question)
    return {"generation": result.content}

#### 웹 검색 기반의 답변 가능 여부 판단

In [17]:
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

def web_search(state: AgentState) -> AgentState:
    """
    Perform web search and evaluate results.
    """
    question = state["question"]

    search_tool = TavilySearchResults(max_results=3)
    search_results = search_tool.invoke(question)

    class answer_available(BaseModel):
        """Binary score for answer availability."""
        binary_score: str = Field(description="""
                                    If web search result can solve the user's ask, answer 'yes'.
                                    If not, answer 'no'""")
    
    evaluator = llm.with_structured_output(answer_available)
    eval_prompt = ChatPromptTemplate.from_messages([
        ("system", "Evaluate if these search results can answer the user's question with a simple yes/no."),
        ("user", """
         Question: {question}
         Seach Result: {results}
         Can these results answer the question adequately?
         """)
    ])

    print("--- Check whether web search is sufficient for user's ask ---")

    evaluation = evaluator.invoke(
        eval_prompt.format(
            question=question, results="\n".join(f"- {result['content']}" for result in search_results)
        )
    )

    return {
        "search_results": search_results,
        "web_score": evaluation.binary_score
    }
    


#### 웹 검색으로 해결 가능한지 여부 판단

In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

question = "autorag github에서 명시된 설치 방법을 알려줘"

result = web_search({"question": question})
print(result)


--- Check whether web search is sufficient for user's ask ---
{'search_results': [{'url': 'https://github.com/Marker-Inc-Korea/AutoRAG/blob/main/docs/source/install.md', 'content': 'To install AutoRAG, you can use pip:\n\nPlus, it is recommended to install PyOpenSSL and nltk libraries for full features.\n\nNote for Windows Users\n\nAutoRAG is not fully supported on Windows yet. There are several constraints for Windows users.\n\nDue to the constraints, we recommend using Docker images for running AutoRAG on Windows.\n\nPlus, you MAKE SURE UPGRADE UP TO v0.3.1 for Windows users.\n\nInstallation for Local Models 🏠 [...] For using local models, you need to install some additional dependencies.\n\nInstallation for Parsing 🌲\n\nFor parsing you need to install some local packages like libmagic,\ntesseract, and poppler.\nThe installation method depends upon your OS.\n\nAfter installing this, you can install AutoRAG with parsing like below.\n\nInstallation for Korean 🇰🇷\n\nYou can install opti

#### 웹 검색 결과로 해결 가능/불가능 여부로 다음 노드 라우팅하는 함수 정의

In [19]:
def route_after_search(state: AgentState) -> Literal["generate", "github_search"]:
    """
    Route based on search evaluation.
    """
    if state["web_score"] == "yes":
        print("--- 웹검색 결과로 해결 가능 ---")
        return "web_generate"
    else:
        print("--- 웹검색 결과로 해결 불가, 깃허브 검색 진행 ---")
        return "github_search"

#### 웹검색 기반 답변 노드

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def web_generate(state: AgentState):
    question = state["question"]
    web_results = state["search_results"]

    def format_web_results(results):
        formatted = []
        for i, result in enumerate(results, 1):
            formatted.append(f"Source {i}: \nURL: {result['url']}\nContent: {result['content']}\n")
        return "\n".join(formatted)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant that generates comprehensive answers based on web search results.
        Use the provided search results to answer the user's question.
        Make sure to synthesize information from multiple sources when possible.
        If the search results don't contain enough information to fully answer the question, acknowledge this limitation."""),
        ("user", """Question: {question}

        Search Results:
        {web_results}

        Please provide a detailed answer based on these search results. Answer in Korean""")
    ])
    chain = (
        {
            "question": lambda x: x["question"], 
            "web_results": lambda x: format_web_results(x["search_results"])
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    print("--- 웹 검색 결과 기반 답변 생성중 ---")
    response = chain.invoke({
      "question": question,
      "web_results": web_results  
    })

    return {"generation": response}


#### 깃헙 레포 정보를 가져오는 함수

In [24]:
from langchain_community.document_loaders import GithubFileLoader
from chromadb.config import Settings
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

def git_loader(repo, branch_name):
    loader = GithubFileLoader(
        repo=repo,
        branch=branch_name,
        access_token=GIT_TOKEN,
        github_api_url="https://api.github.com",
        file_filter=lambda file_path: file_path.endswith(".md"),    # 내가 가져올 파일 확장자자
    )

    documents = loader.load()

    return documents

def git_vector_embedding(repo_name):
    client = chromadb.Client(Settings(
        is_persistent=True,
        persis_directory="./chroma_db"  # 저장될 디렉토리 지정
    ))

    collection_name = repo_name.split("/")[1]

    existing_collections = client.list_collections()
    if collection_name in [col.name for col in existing_collections]:
        print(f"--- Loading existing collection for {collection_name} ---")
        vectorstore = Chroma(
            client=client,
            collection_name=collection_name,
            embedding_function=OpenAIEmbeddings(),
        )
    else:
        print(f"--- Creating new collection for {collection_name} ---")
        try:
            git_docs = git_loader(repo_name, "master")
        except:
            git_docs = git_loader(repo_name, "main")

        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=500, chunk_overlap=50
        )
        docs_splits = text_splitter.split_documents(git_docs)

        vectorstore = Chroma.from_documents(
            documents=docs_splits,
            collection_name=collection_name,
            embedding=OpenAIEmbeddings(),
            client=client
        )

    return vectorstore
